# Fases de entrenamiento de una red neuronal

Importamos los datos de un corpus en español.

Corpus de español:
- AnCora | Github: https://github.com/UniversalDependencies/UD_Spanish-AnCora

- usamos el conllu parser para leer el corpus: https://pypi.org/project/conllu/ . Es muy usado, se puede trabajar con pip install conllu

- Etiquetas Universal POS (Documentación): https://universaldependencies.org/u/pos/ . Hay referencias universales para clasificar palabras


In [2]:
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
fatal: destination path 'UD_Spanish-AnCora' already exists and is not an empty directory.


In [3]:
#leer los datos que vienen de un archivo conllu
from conllu import parse_incr
wordlist = []#creo una lista vacía
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") #abro el archivo que descargué al clonar el repositorio, agrego permisos de lectura con "r"

#for tokenlist in parse_incr(data_file):
#    print(tokenlist.serialize())
    
    #quiero ver los tokens
    #serialize es un atributo propio del formato conllu, sirve para poder ver bien los tokens


Veamos la estructura de los tokens

In [ ]:
print(tokenlist)
print(tokenlist[1])

TokenList<Lo, cierto, es, que, a, mí, ,, me, da, un, poco, de, pena, .>
cierto


In [ ]:
tokenlist[1]['form']+'|'+tokenlist[1]['upos']# el atributo form es una palabra, upos es la categoría gramatical.

'cierto|ADJ'

In [ ]:
for i in range(10):
    print(tokenlist[i]['form']+'|'+tokenlist[i]['upos'])

Lo|PRON
cierto|ADJ
es|AUX
que|SCONJ
a|ADP
mí|PRON
,|PUNCT
me|PRON
da|VERB
un|DET


# Entrenamiento del modelo- Calculo de conteos
La primera etapa es el cálculo de conteos

- tags(tags) ``tagCountDict``: $C(tag)$
- emisiones(word|tag) ``emissionProbDict``: $C(word|tag)$
- transiciones(tag|prevtag) ``transitionDict``: $C(tag|prevtag)$

In [ ]:
#tenemos tres diccionarios inicialmente vacíos

tagCountDict = {}
emissionDict = {}
transitionDict = {}

tagtype = 'upos'
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") #abro el archivo que descargué al clonar el repositorio, agrego permisos de lectura con "r"
for tokenlist in parse_incr(data_file):
    prevtag=None
    for token in tokenlist:
        #C(tag) el conteo de las etiquetas
        tag = token[tagtype]
        if tag in tagCountDict.keys():
            tagCountDict[tag] +=1
        else:
            tagCountDict[tag] = 1

        #C(WORD|tag) -> probabilidades de emisión
        wordtag = token['form'].lower()+'|'+token[tagtype] #(palabra|etiqueta)
        if wordtag in emissionDict.keys():
            emissionDict[wordtag] +=1
        else:
            emissionDict[wordtag] = 1

        #Ahora vamos a hacer un conteo para las probabilidades de transición
        #C(tag|tag_previo)
        if prevtag is None:
            prevtag=tag
            continue
        transitiontags = tag+'|'+prevtag
        if transitiontags in transitionDict.keys():
            transitionDict[transitiontags] = transitionDict[transitiontags] + 1
        else:
            transitionDict[transitiontags] = 1
        prevtag = tag    


# Cálculo de probabilidades

- Probabilidades de transición
$$
P(tag|prevtag)=\frac{C(prevtag,tag)}{C(prevtag)}
$$


- Probabilidades de emisión

$$
P(word|tag)=\frac{C(word,tag)}{C(tag)}
$$

In [ ]:
transitionProbDict = {} # matriz A
emissionProbDict = {} # matriz B

# transition Probabilities 
for key in transitionDict.keys():
    tag, prevtag = key.split('|')
    if tagCountDict[prevtag]>0:
        transitionProbDict[key] = transitionDict[key]/(tagCountDict[prevtag])
    else:
        print(key)

# emission Probabilities 
for key in emissionDict.keys():
    word, tag = key.split('|')
    if emissionDict[key]>0:
        emissionProbDict[key] = emissionDict[key]/tagCountDict[tag]
    else:
        print(key)




In [ ]:
#Ya tenemos el modelo, ahora tenemos que guardarlo
import numpy as np
np.save('transitionHMM.npy', transitionProbDict)
np.save('emissionHMM.npy',  emissionProbDict)

#si queremos volver a cargar el modelo ya entrenado
transitionProbDict = np.load('transitionHMM.npy', allow_pickle='TRUE').item()
transitionProbDict['ADJ|ADJ']

0.030225988700564973

# Algoritmo de Viterbi

Se trata de hallar la probabilidad de viterbi $v_t(j)$ en cada elemento $j$ de una cadena de palabras.

$$
v_t(j) = \max_i{(v_{t-1}(i) \times C_{ij}\times P(palabra|j))}
$$

Para el primer elemento, la probabilidad de Viterbi está dada por
$$
v_1(j)= \rho_j^{(0)} \times P(palabra|j)
$$

donde $\rho_j^{(0)}$ es la probabilidad de encontrar la categoría gramatical $j$.


In [ ]:
#identificamos las categorías gramaticales 'upos' únicas en el corpus

stateSet = set([w.split('|')[1] for w in list(emissionProbDict.keys())])

In [ ]:
#A cada categoría asignamos un índice entero 

tagStateDict = {}
for i, state in enumerate(stateSet):
    tagStateDict[state] = i

tagStateDict

In [ ]:
#Calculamos distribución inicial de estados
wordlist = []#creo una lista vacía
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") #abro el archivo que descargué al clonar el repositorio, agrego permisos de lectura con "r"

count = 0 # contador de la longitud del corpus

initTagStateProb = {} #\rho_i^0
for tokenlist in parse_incr(data_file):
    count += 1
    tag = tokenlist[0]['upos']
    if tag in initTagStateProb.keys():
        initTagStateProb[tag] +=1
    else:
        initTagStateProb[tag]=1

for key in initTagStateProb.keys():
    initTagStateProb[key] /= count

initTagStateProb


{'DET': 0.3633615477629988,
 'PROPN': 0.1124546553808948,
 'ADP': 0.16384522370012092,
 'PRON': 0.034461910519951636,
 'SCONJ': 0.02418379685610641,
 'ADV': 0.06287787182587666,
 'PUNCT': 0.07799274486094317,
 'VERB': 0.04353083434099154,
 'ADJ': 0.010882708585247884,
 'CCONJ': 0.03325272067714631,
 'NOUN': 0.02720677146311971,
 '_': 0.0006045949214026602,
 'INTJ': 0.0006045949214026602,
 'AUX': 0.022370012091898428,
 'NUM': 0.01995163240628779,
 'SYM': 0.0006045949214026602,
 'PART': 0.0018137847642079807}

In [ ]:
#Verificamos si la suma de las probabilidades nos da uno

np.array([initTagStateProb[k] for k in  initTagStateProb.keys()]).sum()

1.0

# Construcción del algoritmo de Viterbi

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def Viterbitags(secuencia, transitionProbDict = transitionProbDict, emissionProbDict = emissionProbDict, tagStateDict= tagStateDict, initTagStateProb=initTagStateProb):
    #inicialización de la primera columna

    seq = word_tokenize(secuencia)
    viterbiProb = np.zeros((17, len(seq)))

    # para la primera columna
    for key in tagStateDict.keys():
        tag_row = tagStateDict[key]
        word_tag= seq[0].lower()+'|'+key
        if word_tag in emissionProbDict.keys():
            viterbiProb[tag_row, 0 ] = initTagStateProb[key]*emissionProbDict[word_tag]

    # para las siguientes columnas
    for col in range(1, len(seq)):
        for key in tagStateDict.keys():
            tag_row = tagStateDict[key]
            word_tag= seq[col].lower()+'|'+key
            if word_tag in emissionProbDict.keys():
                possible_probs = []
                for key2 in tagStateDict.keys():
                    tag_row2 = tagStateDict[key2]
                    tag_prevtag = key+'|'+key2
                    if tag_prevtag in transitionProbDict.keys():
                        if viterbiProb[tag_row2, col-1]>0:
                            possible_probs.append(
                                viterbiProb[tag_row2, col-1]*transitionProbDict[tag_prevtag]*emissionProbDict[word_tag])
                viterbiProb[tag_row, col] = max(possible_probs)

    #construccion de la sección de tags
    res= []
    for i, p in enumerate(seq):
        for tag in tagStateDict.keys():
            if tagStateDict[tag] == np.argmax(viterbiProb[:, i]):
                res.append((p,tag))

    return res

matrix = Viterbitags('el mundo es pequeño')
matrix




[('el', 'DET'), ('mundo', 'NOUN'), ('es', 'AUX'), ('pequeño', 'ADJ')]

# Entrenamiento directo con NLTK


In [ ]:
import nltk
nltk.download('treebank')
from nltk.corpus import treebank
train_data = treebank.tagged_sents()[:3900]  #este dataset ya está tageado



[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [ ]:
train_data

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]

In [ ]:
# Vamos a usr directamente el algoritmo de nltk para entrenar

from nltk.tag import hmm
tagger = hmm.HiddenMarkovModelTrainer().train_supervised(train_data)



In [ ]:
tagger.tag("Pierre Vinken will get old".split())

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 ('will', 'MD'),
 ('get', 'VB'),
 ('old', 'JJ')]

In [ ]:
tagger.evaluate(train_data)

0.9815403947224078

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=31a57299-7db0-417b-9a80-6e17fdb92497' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>